# Introducing OpenManipulator-X
Wir werden hier die Grundlegenden Funktionalitäten des OpenManipulator-X Roboterarms kennenlernen. Normalerweiße gibt es bereits eine Implementierung für die Vorwärts und Rückwärtskinemtic des Openmanipulator, der Roboter wird in der Gazebo-Umgebung simuliert und die Bewegung des Roboters mit Hilfe von ROS angesteuert. 

In den Übungen werden wir allerdings solbst die Kinematik implementieren und den Roberarm mithilfe von `MeshCatMechanisms.jl` und `RigidBodyDynamics.jl` simulieren und steuern.

## Installation

Zum start müssen wir noch die folgenden Pakete laden und installieren. Die Installation der Pakete kann einige Zeit in Anspruch nehmen. Ist müsst sie aber nur einmal durchführen.

> Vergesst nicht, dass ihr den Julia-Kernel für dieses Notebook auswählt. Er sollte in ```/snap/bin/julia``` sein

In [13]:
import Pkg
# Pkg.generate("OpenMEnv") # generate a new package
Pkg.activate("OpenMEnv") # activate the package
Pkg.add("RigidBodyDynamics")
Pkg.add("MeshCatMechanisms")
Pkg.add("MeshCat")
Pkg.add("LinearAlgebra")
Pkg.add("StaticArrays")

  Activating project at `~/Library/Mobile Documents/com~apple~CloudDocs/Projects/IngGru_numerik/SS2024/Excercise/Exercise 3/OpenMEnv`
   Resolving package versions...
  No Changes to `~/Library/Mobile Documents/com~apple~CloudDocs/Projects/IngGru_numerik/SS2024/Excercise/Exercise 3/OpenMEnv/Project.toml`
  No Changes to `~/Library/Mobile Documents/com~apple~CloudDocs/Projects/IngGru_numerik/SS2024/Excercise/Exercise 3/OpenMEnv/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Library/Mobile Documents/com~apple~CloudDocs/Projects/IngGru_numerik/SS2024/Excercise/Exercise 3/OpenMEnv/Project.toml`
  No Changes to `~/Library/Mobile Documents/com~apple~CloudDocs/Projects/IngGru_numerik/SS2024/Excercise/Exercise 3/OpenMEnv/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Library/Mobile Documents/com~apple~CloudDocs/Projects/IngGru_numerik/SS2024/Excercise/Exercise 3/OpenMEnv/Project.toml`
  No Changes to `~/Library/Mobile Documents/com~apple~CloudDocs/Projec

## Environment Initialisierung

Als nächstes müssen wir das Environment initialisieren. Dazu müssen wir die folgenden Zeilen ausführen. 

In [14]:
import Pkg
Pkg.activate("OpenMEnv") 
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using LinearAlgebra
using StaticArrays

  Activating project at `~/Library/Mobile Documents/com~apple~CloudDocs/Projects/IngGru_numerik/SS2024/Excercise/Exercise 3/OpenMEnv`


## Laden der OpenManipulator-X Beschreibung

Unter `./open_manipulator_description/urdf/` findet ihr die URDF-Datei des OpenManipulator-X. Diese Datei wird in den folgenden Schritten geladen und zur visualisierung verwendet. 

In [4]:
srcdir = "../open_manipulator_description/urdf/"
urdf = joinpath(srcdir, "open_manipulator.urdf")
mechanism = parse_urdf(urdf)

Spanning tree:
Vertex: world (root)
  Vertex: link2, Edge: joint1
    Vertex: link3, Edge: joint2
      Vertex: link4, Edge: joint3
        Vertex: link5, Edge: joint4
          Vertex: gripper_link, Edge: gripper
          Vertex: gripper_link_sub, Edge: gripper_sub
No non-tree joints.

Wie ihr seht besteht der Roboterarm aus einem `tree` mit `vertices` und `edges`. Vergleicht die Ausgabe mit dem folgenden Bild welches den Roboterarm zeigt.

![Image](./OpenManipulator.png)

## Simulation und Visualisierung

Als nächstes starten wir die Simulation und Visualisierung des Roboters. Dazu müssen wir die folgenden Zeilen ausführen.

In [5]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf));
render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat /Users/damian/.julia/packages/MeshCat/oC0sL/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("link1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x6c, 0x69, 0x6e, 0x6b, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "link2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6f, 0x72, 0x6c, 0x64, 0x2f, 0x6c, 0x69, 0x6e, 0x6b, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_joint1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "link3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x6e, 0x6b, 0x32, 0x2f, 0x6c, 0x69, 0x6e, 0x6b, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_joint2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "link4" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x6e, 0x6b, 0x33, 0x2f, 0x6c, 0x69, 0x6e, 0x6b, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_joint3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x74, 0x65, 0x72, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "link5" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0

Ihr könnt in der Visualisierung mit der Maus navigieren um den Roboterarm von verschiedenen Perspektiven zu betrachten. 

Als nächstes wollen wir den Roboterarm bewegen. Ohne die Kinematik zu implementieren können wir nur die Gelenkwinkel des Roboters ändern. Mit der methode `set_configuration` können wir die Gelenkwinkel des Roboters ändern. Hier können wir 7 Werte anpassen um die Gelenkwinkel zu ändern. Die ersten 4 sind die Winkle der Gelenke von unten nach oben [ID11, ID12, ID13, ID14], der 5. und 6. Wert ist für die Steuerung des Grippers. 

In [6]:
set_configuration!(mvis, [0.0, 1.5, -1.5, 0.0, 0.0, 0.0])

Wenn wir die Winkel ändern, können wir sehen wie sich der Roboterarm bewegt. 

Um den Roboterarm in die Ausgangsposition zu bewegen, können wir die folgenden Werte verwenden: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0].

In [ ]:
set_configuration!(mvis, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

Um den aktuellen Zustand des Roboters zu erhalten, können wir den state der Simulation abfragen.

In [10]:
state = mvis.state

state.q

6-element SegmentedVector{JointID, Float64, Base.OneTo{JointID}, Vector{Float64}}:
  0.0
  1.5
 -1.5
  0.0
  0.0
  0.0

# Motivation zur Kinematik

Was ist wenn wir jetzt aber wissen wollen wo sich der Endeffektor des Roboters befindet, wenn wir die Gelenkwinkel kennen? Der Roboter hat ja ein definiertes Koordinatensystem (https://emanual.robotis.com/docs/en/platform/openmanipulator_x/specification/#hardware-specification) anhand dessen wir die Tranformationen ausrechnen können. Wir bekommen alle informationen die wir benötigen aus der Simulation aber wir müssen dann die Kinematik des Roboters berechnen. Als kleiner vorgeschmack schauen wir uns die `transform_to_root` Methode an. Diese Methode gibt uns die Transformation eines Punktes relativ zum Basis-Koordinatensystem des Roboters an. 

In [12]:
gripper_joint = findjoint(mechanism, "gripper")
transform_to_root(state, frame_after(gripper_joint))

Transform3D from "after_gripper" to "world":
rotation: 0.0 rad about [1.0, 0.0, 0.0], translation: [0.3470770511253438, 0.021, 0.061614482134968666]

Mit der Methode `transform` können wir die Transformation eines Punktes relativ zu einem anderen Punkt berechnen: 

In [16]:
zero_v = zero(SVector{3})
world = findbody(mechanism, "world")

ee = transform(state, Point3D(frame_after(gripper_joint), zero_v), default_frame(world))

Point3D in "world": [0.3470770511253438, 0.021, 0.061614482134968666]

Testet die Methode mit verschiedenen Punkten und schaut ob die Transformationen korrekt sind. In der nächsten Übung werden wir die Kinematic dann selbst implementieren.